In [1]:
using JuMP
using CPLEX
using Distributions
using LinearAlgebra
using Statistics
using Dates
using DataFrames
using SDDP
using Plots
import CSV
using JSON
try
    using Revise
catch e
    @warn "Error initializing Revise" exception=(e, catch_backtrace())
end

In [2]:
includet(pwd() * "\\Water_Regulation\\WaterRegulation.jl")
using .WaterRegulation

### General Parameters

In [3]:
Stages = 8
Hours = 24
PricePoints = [0.0, 1.0]
I = length(PricePoints)-1

println("Stages :" , Stages, "\n Hours: ", Hours, "\n I : ", I, "\n Price Points = ", [i for i in PricePoints])

Stages :8
 Hours: 24
 I : 1
 Price Points = 

[0.0, 1.0]


In [4]:
filepath_Ljungan = pwd() * "\\Water_Regulation\\TestDataWaterRegulation\\Ljungan.json"
filepath_prices = pwd() *  "\\Data\\Spot Prices\\prices_df.csv"
filepath_results = pwd() * "\\Results\\LambdaZero\\"
all_res, plants, parts = read_data(filepath_Ljungan)

(Reservoir[Reservoir with name: Flasjön
, Reservoir with name: Holmsjön
], HydropowerPlant[Power Plant: Flasjö
, Power Plant: Trangfors
, Power Plant: Rätan
, Power Plant: Turinge
, Power Plant: Bursnäs
, Power Plant: Järnvägsforsen
, Power Plant: Parteboda
, Power Plant: Hermansboda
, Power Plant: Ljunga
, Power Plant: Nederede
, Power Plant: Skallböle
, Power Plant: Matfors
, Power Plant: Viforsen
], Participant[Name: Sydkraft
, Name: Fortum
, Name: Statkraft
])

### Have a look at the river system we are dealing with    
We will work with Sydkraft in this example

In [5]:
j = parts[1]

Name: Sydkraft


In [6]:
res = filter(r -> j.participationrate[r] > 0, all_res)

1-element Vector{Reservoir}:
 Reservoir with name: Flasjön


In [7]:
j.plants

5-element Vector{HydropowerPlant}:
 Power Plant: Flasjö

 Power Plant: Trangfors

 Power Plant: Rätan

 Power Plant: Turinge

 Power Plant: Bursnäs


### Define the SDDP Model

### Set Parameters necessary for Input into Model

In [8]:
Qref = Dict{Reservoir, Float64}(r => 1.0 for r in res)
scenario_count = 2
Prices = [floor.(rand(Hours), sigdigits=3) for i in 1:scenario_count]
Inflows = [0.0]
Omega = [(price = p, inflow = v) for p in Prices for v in Inflows]
P = [1/length(Omega) for om in Omega]
# StartUp Constraints
S = 0.1
mu_up = 1
mu_down = 1

1

In [9]:
function subproblem_builder(subproblem::Model, node::Int)
    # State Variables
    @variables(subproblem, begin
        0 <= l_real[r = res] <= r.maxvolume, (SDDP.State, initial_value = r.currentvolume)
        l_ind[r = res], (SDDP.State, initial_value = r.currentvolume)
        Qnom[r = res], (SDDP.State, initial_value = 0)
        x[i = 1:I+1, t = 1:Hours], (SDDP.State, initial_value = 0)
    end)
    # Control Variables
    @variables(subproblem, begin
        z_up[t = 1:Hours] >= 0
        z_down[t = 1:Hours] >= 0
        delta_start[k = j.plants, t = 1:Hours], (Bin)
        delta_ind[r = res], (Bin)
        u[k = j.plants, t = 1:Hours], (Bin)
        w[k = j.plants, t = 1:Hours]
        Qnom_change[r = res] >= 0
        x_change[i = 1:I+1, t = 1:Hours] >= 0
        Qreal[r = res, t = 1:Hours] >= 0
        y[t = 1:Hours] >= 0
    end)
    # Random Variables
    @variables(subproblem, begin
        Qinflow[r = res]
    end)
    if node == 1
        # Transition Function
        for r in res
            # Real and Individual Reservoir Balance
            @constraint(subproblem, l_real[r].out == l_real[r].in)
            @constraint(subproblem, l_ind[r].out == l_ind[r].in)
            @constraint(subproblem, Qnom[r].out == Qnom_change[r])
        end
        for t in 1:Hours
            for i in 1:I
                @constraint(subproblem, x[i,t].out == x_change[i,t])
                @constraint(subproblem, x_change[i,t] <= x_change[i+1,t])
            end
            @constraint(subproblem, x[I+1,t].out == x_change[I+1,t])
        end
        # Constraints
        for r in res
            @constraint(subproblem, Hours * Qnom_change[r] <= l_real[r].in)
            # Negative individual Reservoir Complications
            @constraint(subproblem, delta_ind[r] => {Qnom_change[r] <= Qref[r]})
            @constraint(subproblem, !delta_ind[r] => {0 <= l_ind[r].in})
        end
        @stageobjective(subproblem, 0)
    else
        # Transition Function
        for r in res
            # Real and Individual Reservoir Balances
            @constraint(subproblem, l_real[r].out == l_real[r].in - Hours * (Qnom[r].in - Qinflow[r]))
            @constraint(subproblem, l_ind[r].out == l_ind[r].in - Hours * (Qnom[r].in - Qref[r]))
            @constraint(subproblem, Qnom[r].out == Qnom_change[r])
        end
        for t in 1:Hours
            for i in 1:I+1
                @constraint(subproblem, x[i,t].out == x_change[i,t])
            end
        end
        # Constraints
        for t in 1:Hours
            for i in 1:I
                # Increasing bidding Curve constraint
                @constraint(subproblem, x_change[i,t] <= x_change[i+1,t])
            end
            # Fulfill Demand Constraint
            @constraint(subproblem, y[t] == sum(w[k,t] for k in j.plants) + z_up[t] - z_down[t])
            for k in j.plants
                # Power Generation Constraint
                @constraint(subproblem, w[k,t] <= sum(Qreal[r,t] for r in find_us_reservoir(k.reservoir)) * k.equivalent)
                # On-Off and Maximum Generation Constraint
                @constraint(subproblem, u[k,t] * 0 <= w[k,t])
                @constraint(subproblem, w[k,t] <= u[k,t] * 0.5 * k.equivalent * k.spill_reference_level)
                if t > 1
                    # Start-Up
                    @constraint(subproblem, delta_start[k, t] >= u[k,t] - u[k,t-1])
                end
            end
        end

        for r in res
            # Average Discharge Constraint
            @constraint(subproblem, sum(Qreal[r,t] for t in 1:Hours) == Qnom[r].in)
            # Negative Individual Reservoir Complications
            @constraint(subproblem, delta_ind[r] => {Qnom_change[r] <= Qref[r]})
            @constraint(subproblem, !delta_ind[r] => {0 <= l_ind[r].in})
        end
        # Parametrize Uncertainty
        SDDP.parameterize(subproblem, Omega, P) do om
            for r in res
                JuMP.fix(Qinflow[r], om.inflow)
            end
            for t in 1:Hours
                for i in 1:I
                    if (om.price[t] <= PricePoints[i+1]) && (om.price[t] >= PricePoints[i])
                        # Market Clearing Constraint
                        @constraint(subproblem, y[t] == ((om.price[t] - PricePoints[i])/(PricePoints[i+1] - PricePoints[i]))*x[i+1,t].in + ((PricePoints[i+1] - om.price[t])/(PricePoints[i+1] - PricePoints[i]))*x[i,t].in)
                    end
                end
            end
            # Stage-objective
            @stageobjective(
                #subproblem, sum(y[t] * om.price[t] - sum(S * delta_start[k, t] for k in j.plants) - (mu_up * z_up[t] + mu_down * z_down[t]) for t in 1:Hours)
                subproblem, sum(y[t] * om.price[t]  for t in 1:Hours)
                )
        end
    end
    return
end


subproblem_builder (generic function with 1 method)

In [10]:
for i in 1:I
    println(PricePoints[i])
end
for el in Omega
    println(el)
end


0.0


(price = [0.474, 0.224, 0.853, 0.077, 0.264, 0.0775, 0.727, 0.504, 0.433, 0.942, 0.662, 0.645, 0.878, 0.807, 0.844, 0.382, 0.701, 0.256, 0.281, 0.574, 0.488, 0.0589, 0.863, 0.356], inflow = 0.0)
(price = [

0.0331, 0.65, 0.157, 0.486, 0.905, 0.317, 0.937, 0.656, 0.0507, 0.66, 0.857, 0.275, 0.456, 0.131, 0.991, 0.409, 0.517, 0.837, 0.33, 0.999, 0.296, 0.126, 0.222, 0.591], inflow = 0.0)


In [11]:
model = SDDP.LinearPolicyGraph(
    subproblem_builder;
    stages = 8,
    sense = :Max,
    upper_bound = 1e3,
    optimizer = CPLEX.Optimizer
)

A policy graph with 8 nodes.
 Node indices: 1, 2, 3, 4, 5, 6, 7, 8


### Train the model

In [12]:
SDDP.train(model; iteration_limit = 1, duality_handler = SDDP.LagrangianDuality())

-------------------------------------------------------------------
         SDDP.jl (c) Oscar Dowson and contributors, 2017-23
-------------------------------------------------------------------


problem
  nodes           : 8
  state variables : 51
  scenarios       : 1.28000e+02
  existing cuts   : false


options
  solver          : serial mode
  risk measure    : 

SDDP.Expectation()
  sampling scheme : SDDP.InSampleMonteCarlo
subproblem structure
  VariableRef                                                                   : [610, 610

]
  AffExpr in MOI.EqualTo{Float64}                                               : [51, 76]
  AffExpr in MOI.GreaterThan{Float64}                                           : [115, 115]
  AffExpr in MOI.LessThan{Float64}                                              : [25, 384]
  VariableRef in MOI.GreaterThan{Float64}                                       : [146, 147]
  VariableRef in MOI.LessThan{Float64}                                          : [2, 2]
  VariableRef in MOI.ZeroOne                                                    : [241, 241]


  Vector{AffExpr} in MOI.Indicator{MOI.ACTIVATE_ON_ONE, MOI.LessThan{Float64}}  : [1, 1]
  Vector{AffExpr} in MOI.Indicator{MOI.ACTIVATE_ON_ZERO, MOI.LessThan{Float64}} : [1, 1]


numerical stability report
  matrix range     [1e-03, 2e+01]
  objective range  [3e-02, 1e+00]
  bounds range     [1e+03, 4e+04]
  rhs range        [2e+01, 2e+01]
-------------------------------------------------------------------
 iteration    simulation      bound        time (s)     solves  pid


-------------------------------------------------------------------


┌ Info: Writing cuts to the file `model.cuts.json`
└ @ SDDP C:\Users\Lenni\.julia\packages\SDDP\ppkik\src\algorithm.jl:287


ErrorException: Unable to retrieve solution from node 8.

  Termination status : INFEASIBLE
  Primal status      : NO_SOLUTION
  Dual status        : NO_SOLUTION.

The current subproblem was written to `subproblem_8.mof.json`.

There are two common causes of this error:
  1) you have a mistake in your formulation, or you violated
     the assumption of relatively complete recourse
  2) the solver encountered numerical issues

See https://odow.github.io/SDDP.jl/stable/tutorial/warnings/ for more information.

### Obtain the Bidding and Nomination Decisions

In [13]:
rule = SDDP.DecisionRule(model; node = 1)
solution = SDDP.evaluate(
    rule;
    incoming_state = merge(Dict(Symbol("l_real[$(r)]") => r.currentvolume for r in res), Dict(Symbol("l_ind[$(r)]") => r.currentvolume for r in res)),
    controls_to_record = [:Qnom, :y, :z_up, :z_down]
)


(stage_objective = 0, outgoing_state = Dict(Symbol("x[2,24]") => 0.0, Symbol("x[2,8]") => 0.0, Symbol("x[1,4]") => 0.0, Symbol("x[2,4]") => 0.0, Symbol("x[1,24]") => 0.0, Symbol("x[2,6]") => 0.0, Symbol("x[2,21]") => 0.0, Symbol("x[1,7]") => 0.0, Symbol("x[1,17]") => 0.0, Symbol("x[1,5]") => 0.0…), controls = Dict{Symbol, AbstractVector}(:Qnom => 1-dimensional DenseAxisArray{SDDP.State{Float64},1,...} with index sets:
    Dimension 1, Reservoir[Reservoir with name: Flasjön
]
And data, a 1-element Vector{SDDP.State{Float64}}:
 SDDP.State{Float64}(0.0, 0.0), :y => [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], :z_down => [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], :z_up => [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]))